In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

def align_timestamps(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors="coerce")
    df[col] = df[col].dt.tz_localize(None)
    return df


def to_timeseries(df, id_col, time_col, target_col):
    df = align_timestamps(df, time_col)
    return TimeSeriesDataFrame.from_data_frame(
        df[[id_col, time_col, target_col]],
        id_column=id_col,
        timestamp_column=time_col,
    )

def to_df(obj):
    return obj if isinstance(obj, pd.DataFrame) else pd.DataFrame(obj)


def bootstrap_metrics(actual, predicted, n_bootstraps=1000, alpha=0.05):
    actual, predicted = np.array(actual), np.array(predicted)
    n = len(actual)
    rng = np.random.default_rng(42)

    def compute_metrics(a, p):
        mae = np.mean(np.abs(a - p))
        rmse = np.sqrt(np.mean((a - p) ** 2))
        mse = np.mean((a - p) ** 2)
        mape = np.mean(np.abs((a - p) / np.maximum(np.abs(a), 1e-8))) * 100
        smape = 100 * np.mean(2 * np.abs(a - p) / (np.abs(a) + np.abs(p) + 1e-8))
        return mae, rmse, mse, mape, smape

    base = compute_metrics(actual, predicted)
    boot = np.zeros((n_bootstraps, len(base)))
    for i in range(n_bootstraps):
        idx = rng.integers(0, n, n)
        boot[i, :] = compute_metrics(actual[idx], predicted[idx])
    lower = np.percentile(boot, 100 * (alpha / 2), axis=0)
    upper = np.percentile(boot, 100 * (1 - alpha / 2), axis=0)
    names = ["mae", "rmse", "mse", "mape", "smape"]
    return {
        **dict(zip(names, base)),
        **{f"{k}_ci_lower": l for k, l in zip(names, lower)},
        **{f"{k}_ci_upper": u for k, u in zip(names, upper)},
    }

def nested_cv_autogluon(
    resampled_df,
    prediction_length,
    value_to_predict,
    resample_rate,
    metric="MAE",
    n_outer_folds=3,
):
    encounter_ids = resampled_df["encounter_id"].unique()
    outer_kf = KFold(n_splits=n_outer_folds, shuffle=True, random_state=42)

    aggregated_actuals, aggregated_predictions = [], []
    fold_results = []

    print(f"Starting Nested {n_outer_folds}-Fold Cross-Validation with AutoGluon...")

    for fold_num, (train_idx, test_idx) in enumerate(outer_kf.split(encounter_ids)):
        print(f"\n--- Outer Fold {fold_num + 1}/{n_outer_folds} ---")

        train_ids = encounter_ids[train_idx]
        test_ids = encounter_ids[test_idx]

        outer_train_df = resampled_df[resampled_df["encounter_id"].isin(train_ids)].copy()
        outer_test_df = resampled_df[resampled_df["encounter_id"].isin(test_ids)].copy()

        # Convert to TimeSeriesDataFrame
        train_tsf = to_timeseries(outer_train_df, "encounter_id", "recorded_time", value_to_predict)


        predictor = TimeSeriesPredictor(
            prediction_length=prediction_length,
            path=f"autogluon_outer_fold{fold_num+1}",
            target=value_to_predict,
            eval_metric=metric,
            freq=resample_rate,
            verbosity=2,
            quantile_levels=[0.1, 0.5, 0.9],
        )

        predictor.fit(
            train_tsf,
            #presets="best_quality",
            num_val_windows=1,
            time_limit=15000,
            hyperparameters={
                        "AutoARIMA":{"n_jobs":3,"max_ts_length":14},
                        "NaiveModel":{"n_jobs":3,"max_ts_length":14},
                        "AverageModel":{"n_jobs":3,"max_ts_length":14},

    
        },
            enable_ensemble=False,
        )


        history_list = []
        target_list = []
        for eid, group in outer_test_df.groupby("encounter_id"):
            group = group.sort_values("recorded_time")
            if len(group) <= prediction_length:
                continue  # skip too-short series
            hist = group.iloc[:-prediction_length]
            targ = group.iloc[-prediction_length:]
            history_list.append(hist)
            target_list.append(targ)

        test_history_df = pd.concat(history_list)
        test_target_df = pd.concat(target_list)

        test_history_tsf = to_timeseries(test_history_df, "encounter_id", "recorded_time", value_to_predict)

     
        forecast = predictor.predict(test_history_tsf)
        forecast_df = to_df(forecast).reset_index()

   
        forecast_df["timestamp"] = pd.to_datetime(forecast_df["timestamp"]).dt.tz_localize(None)
        test_target_df["recorded_time"] = pd.to_datetime(test_target_df["recorded_time"]).dt.tz_localize(None)

        merged = forecast_df.merge(
            test_target_df,
            left_on=["item_id", "timestamp"],
            right_on=["encounter_id", "recorded_time"],
            how="inner",
        )

        actuals = merged[value_to_predict].values
        preds = merged["mean"].values

        metrics = bootstrap_metrics(actuals, preds)
        fold_results.append(metrics)
        print(f"Outer Fold {fold_num+1} MAE: {metrics['mae']:.4f}")

        aggregated_actuals.extend(actuals)
        aggregated_predictions.extend(preds)

 
    final_metrics = bootstrap_metrics(np.array(aggregated_actuals), np.array(aggregated_predictions))


    print("Final Nested CV Results (AutoGluon TimeSeries):")

    print(f"MAE:   {final_metrics['mae']:.4f} (95% CI: [{final_metrics['mae_ci_lower']:.4f}, {final_metrics['mae_ci_upper']:.4f}])")
    print(f"RMSE:  {final_metrics['rmse']:.4f} (95% CI: [{final_metrics['rmse_ci_lower']:.4f}, {final_metrics['rmse_ci_upper']:.4f}])")
    print(f"MSE:   {final_metrics['mse']:.4f} (95% CI: [{final_metrics['mse_ci_lower']:.4f}, {final_metrics['mse_ci_upper']:.4f}])")
    print(f"MAPE:  {final_metrics['mape']:.2f}% (95% CI: [{final_metrics['mape_ci_lower']:.2f}%, {final_metrics['mape_ci_upper']:.2f}%])")
    print(f"sMAPE: {final_metrics['smape']:.2f}% (95% CI: [{final_metrics['smape_ci_lower']:.2f}%, {final_metrics['smape_ci_upper']:.2f}%])")

    return final_metrics, fold_results, predictor



if __name__ == "__main__":
    print("Generating dummy resampled_df for demo. Pls replace with your actual data.")
    np.random.seed(42)
    encounter_ids = [f"E/{i}" for i in range(6)]
    dfs = []
    for eid in encounter_ids:
        times = pd.date_range("2024-01-01", periods=20, freq="D")
        values = np.random.rand(20) * 10
        dfs.append(pd.DataFrame({"encounter_id": eid, "recorded_time": times, "value": values}))
    resampled_df = pd.concat(dfs)

    metrics, fold_results, predictor = nested_cv_autogluon(
        resampled_df=resampled_df,
        prediction_length=1,
        value_to_predict="value",
        resample_rate="D",
        metric="MAE",
        n_outer_folds=10,
    )
